# SUUMO 和歌山県　賃貸物件のスクレイピング

参考URL：https://qiita.com/tomyu/items/a08d3180b7cbe63667c9

> SUUMOからデータを取得

In [1]:
# import
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import numpy as np

In [2]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 114
# SUUMOを和歌山県のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=060&bs=040&ta=30&sc=30201&sc=30202&sc=30203&sc=30204&sc=30205&sc=30206&sc=30207&sc=30208&sc=30209&sc=30300&sc=30340&sc=30360&sc=30380&sc=30400&sc=30420&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}"

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)


> 抽出データの成形・加工

In [3]:
# リストをデータフレームに変換
import pandas as pd
df = pd.DataFrame(data_samples, columns=['カテゴリ', '物件名', '住所', '最寄りのアクセス_1', '最寄りのアクセス_2', '最寄りのアクセス_3', '築年数', '階数', '階', '家賃', '管理費', '敷金', '礼金', '間取', '面積', 'URL'])

In [4]:
df.head(10)

,カテゴリ,物件名,住所,最寄りのアクセス_1,最寄りのアクセス_2,最寄りのアクセス_3,築年数,階数,階,家賃,管理費,敷金,礼金,間取,面積,URL
0,賃貸アパート,レットル・ダムール I,和歌山県日高郡日高町大字荊木,ＪＲ紀勢本線/紀伊内原駅 歩11分,,,築16年,2階建,1階,5.05万円,3600円,-,5.05万円,2DK,45.72m2,https://suumo.jp/chintai/jnc_000085269706/?bc=...
1,賃貸アパート,ＪＲ紀勢本線 藤並駅 2階建 築10年,和歌山県有田郡有田川町大字天満,ＪＲ紀勢本線/藤並駅 歩14分,,,築10年,2階建,2階,4.6万円,2800円,-,4.6万円,1K,30.43m2,https://suumo.jp/chintai/jnc_000085257250/?bc=...
2,賃貸アパート,ＪＲ紀勢本線 藤並駅 2階建 築7年,和歌山県有田郡有田川町大字水尻,ＪＲ紀勢本線/藤並駅 歩8分,,,築7年,2階建,2階,5.75万円,2800円,-,5.75万円,2LDK,57.07m2,https://suumo.jp/chintai/jnc_000085274582/?bc=...
3,賃貸アパート,アニマリート,和歌山県有田郡有田川町大字水尻,ＪＲ紀勢本線/藤並駅 歩8分,,,築7年,2階建,2階,5.75万円,2800円,-,5.75万円,2LDK,57.07m2,https://suumo.jp/chintai/jnc_000085261889/?bc=...
4,賃貸アパート,ＪＲ紀勢本線 藤並駅 2階建 築12年,和歌山県有田郡有田川町大字水尻,ＪＲ紀勢本線/藤並駅 歩4分,,,築12年,2階建,1階,4.5万円,2200円,-,4.5万円,ワンルーム,32.94m2,https://suumo.jp/chintai/jnc_000085257251/?bc=...
5,賃貸アパート,ＪＲ紀勢本線 藤並駅 2階建 築11年,和歌山県有田郡有田川町大字植野,ＪＲ紀勢本線/藤並駅 歩25分,,,築11年,2階建,2階,6万円,3900円,-,6万円,2LDK,57.64m2,https://suumo.jp/chintai/jnc_000085257248/?bc=...
6,賃貸アパート,ＪＲ紀勢本線 藤並駅 2階建 築7年,和歌山県有田郡有田川町大字下津野,ＪＲ紀勢本線/藤並駅 歩33分,,,築7年,2階建,1階,5.6万円,3300円,-,5.6万円,1LDK,50.03m2,https://suumo.jp/chintai/jnc_000085257249/?bc=...
7,賃貸アパート,サンシャイン白馬,和歌山県日高郡日高町大字萩原,ＪＲ紀勢本線/紀伊内原駅 歩5分,ＪＲ紀勢本線/御坊駅 歩44分,紀州鉄道/御坊駅 歩44分,築22年,2階建,2階,4.3万円,3000円,-,-,2DK,43.12m2,https://suumo.jp/chintai/jnc_000085261890/?bc=...
8,賃貸アパート,サンシャイン白馬,和歌山県日高郡日高町大字萩原,ＪＲ紀勢本線/紀伊内原駅 歩5分,ＪＲ紀勢本線/御坊駅 歩44分,紀州鉄道/御坊駅 歩44分,築22年,2階建,2階,4.3万円,3000円,-,-,2DK,43.12m2,https://suumo.jp/chintai/jnc_000078618601/?bc=...
9,賃貸アパート,ボンヌ・シャンス,和歌山県有田郡有田川町大字天満,ＪＲ紀勢本線/藤並駅 歩14分,ＪＲ紀勢本線/紀伊宮原駅 歩51分,ＪＲ紀勢本線/湯浅駅 歩55分,築10年,2階建,2階,4.6万円,2800円,-,4.6万円,1K,30.43m2,https://suumo.jp/chintai/jnc_000085224432/?bc=...


In [5]:
len(df)

11058